Во всех задания требуется написать эффективные реализации, используя функционал numpy. Использовать numba, вызывать код из C или использовать посторонние библиотеки запрещается. В заданиях указаны ориентировочные времена работы. Они могут отличаться на разных машинах, поэтому на вашем компьютере правильное решение может работать как быстрее, так и медленнее. Если в задании указано, что время работы 8 мин, а у вас работает 10, скорее всего, это не страшно. Если же указано 8 мин, а код работает 8 часов, это вряд ли можно списать на медленный компьютер.

# 1. Перебор параметров по сетке

Пусть задана функция вида $$ f(x) = \dfrac{d \log^a{(x)}}{1 + x^b e^{\frac{-c}{x}}} $$
Требуется восстановиться параметры $a, b, c$ и $d$ по значениям функции в узлах заданной сетки. Для этого:
1. Выберете некоторые значения $a, b, c$ и $d$. Например, $a=1.2$, $b=0.4$, $c=1.5$, $d=1.0$;
2. Сгенерируйте сетку, на которой будут вычисляться значение функции. Сетка должна содержать $1,000$ узлов. Например, хорошо подойдет равномерная сетка на отрезке $[1, 100]$ с $1,000$ узлов;
3. Вычислите значения $f(x)$ в узлах сгенерированной сетки;
4. Далее требуется восстановить величины $a, b, c$ и $d$ по значениям $f(x)$ в узлах сгенерированной сетки. Для этого сгенерируйте возможные наборы значений для параметров $a, b, c$ и $d$. Для каждого из параметров должно быть 100 (или 101, если удобнее) значений. Например, для приведенных выше значений хорошо подойдут равномерные сетки на отрезках $[1, 2]$, $[0, 1]$, $[1, 2]$ и $[0, 2]$ соответственно с 101 узлом каждая. Всего имеем $101^4$ комбинаций параметров. Требуется, перебирая всевозможные наборы параметров и вычисления значения функции в узлах сетки, найти те значения, которые будут обеспечивать минимум $l_2$ нормы разности правильного и построенного набора значений.

Ориентировочное время работы при правильной организации перебора $-$ 8 мин.

P.S. В задании требуется просто разумно реализовать перебор на numpy. Придумывать правильный алгоритм перебора, отсечения или как-либо еще "по-умному" производить оптимизацию не требуется.

In [27]:
%%time

#когда нибудь я обязательно докуплю 64гб оперативки ради таких заданий.....

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
from tqdm import tqdm

def f(x, a, b, c, d):
    return d * (np.log(x) ** a) / (1 + (x ** b) * np.exp(-c / x))

x = np.linspace(1, 100, 1000)
a = 1.2
b = 0.4
c = 1.5
d = 1.0

y = f(x, a, b, c, d)


def another_f(x, a, b, c, d):    
    return np.linalg.norm(y[:,None,None,None] - d * (np.log(x) ** a) / (1 + (x ** b) * np.exp(-c / x)), 2, axis=0)


a = np.linspace(1, 2, 100)
b = np.linspace(0, 1, 100)
c = np.linspace(1, 2, 100)
d = np.linspace(0, 2, 100)

# param_list = np.array(np.meshgrid(a, b, c, d)).T.reshape(4,-1)

min_norm = np.linalg.norm(y, 2)
min_index = [0, 0, 0, 0]

for a_i in tqdm(a):
    for index in range(2):
        if index == 0:
            d_new = d[0:50]
        else:
            d_new = d[50:]

        result = another_f(x[:,None,None,None], a_i, b[None,:,None,None], c[None,None,:,None], d_new[None,None,None,:])
        tmp = np.min(result)
        if tmp < min_norm:
            min_norm = tmp
            min_index[0] = np.where(a == a_i)
            min_index[1], min_index[2], min_index[3] = np.where(result == tmp)
            if index == 1:
                min_index[3][0] += 50

print(min_index)

100%|█████████████████████████████████████████████████████████████| 100/100 [08:23<00:00,  5.04s/it]

[(array([14]),), array([39]), array([7]), array([53])]
CPU times: user 5min 4s, sys: 3min 18s, total: 8min 23s
Wall time: 8min 23s


In [29]:
print(np.linalg.norm(y - d[53] * (np.log(x) ** a[14]) / (1 + (x ** b[39]) * np.exp(-c[7] / x)), 2))

0.03888679310095516


# 2. Преобразования тензоров

1. Сгенерируйте случайный тензор (случайный numpy.array) размера $\underbrace{2 \times 2 \times \dots \times 2}_\text{16 раз}$.
2. Последовательно произведите с ним следующую процедуру. Разбейте множители на пары: $$ (2 \times 2) \times (2 \times 2) $$ и вычислите кронекерово произведение векторов в скобках. Повторяйте процедуру пока не останется один вектор. $$ 2 \times 2 \times 2 \times 2 \to (2 \times 2) \times (2 \times 2) \to 4 \times 4 \to (4 \times 4) \to 16 $$ В результате должен получиться вектор длины $2^{16}$.
3. Из построенного вектора попробуйте восстановить исходное представление. А именно, пусть имеется вектор длины $16$. С помощью reshape его можно преобразовать к матрице $4 \times 4$. Для матрицы $4 \times 4$ постройте оптимальное одноранговое приближение - это будет произведение двух векторов размера 4. Для каждого из векторов длины 4 повторите операцию.
$$ 16 \to 4 \times 4 \to (2 \times 2) \times (2 \times 2) \to 2 \times 2 \times 2 \times 2 $$

Схема метода:
<img src="Tensor2-1.png" width="500"/>
<img src="Tensor3-1.png" width="500"/>

(На самом деле это схема для чуть более сложного варианта, значок суммы можно игнорировать)

# 3. Метрика кластеризации

(Задание честно своровано из задач весеннего семестра по курсу машинного обучения)

Требуется реализовать одну из классических метрик для оценки качества кластеризации. Задача кластеризации $-$ это, грубо говоря, задача разбить набор данных на классы похожих. В данном случае объектами будут вектора из $\mathbb{R}^d$, а мерой похожести евклидово расстояние.
<img src="clustering_intro_clustersamplegraph.jpg" width="500"/>

Требуется эффективно реализовать метрику силуэт для оценки качества кластеризации. Для этого необходимо реализовать функцию, которая принимает numpy.array размера $n \times d$, а также список меток размера $n$, где $n$ $-$ число элементов, а $d$ $-$ размерность пространства. Функция должна возвращать одно число $-$ значение метрики.

Суть метрики заключается в оценке двух параметров, характеризующих выделенные кластеры — компактность и отделимость.

Положим, что $C_i$ — номер кластера для объекта $i$.

$s_i$ — компактность кластеризации объекта $i$ определяется как среднее расстояние от него до всех объектов
того же кластера:
$$ s_i = \dfrac{1}{|\{j: C_j = C_i\}| - 1} \sum\limits_{j: C_j=C_i} \|x_i - x_j\| $$

$d_i$ — отделимость кластеризации объекта $i$ определяется как среднее расстояние от него до всех объектов
второго по близости кластера:
$$ \min\limits_{C:C\neq C_i} \dfrac{1}{|\{ j: C_j = C \}|} \sum\limits_{j: C_j=C} \|x_i - x_j\| $$

Тогда силуэт объекта i:
$$ \text{sil}_i = \dfrac{d_i - s_i}{\max{\{d_i, s_i\}}} $$

И, наконец, коэффициент силуэта для выборки определяется как среднее силуэтов объектов:
$$ S = \dfrac{1}{n} \sum\limits_{i} \text{sil}_i $$

На следующем тесте
```
data, labels = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [2.0, 2.0]]),
               np.array([1, 0, 0, 1])
```
Результатом является `-0.15098532303997897`.

На тесте
```
np.random.seed(1568)
data = np.random.randn(5000, 1200)
labels = np.random.randint(low=0, high=100, size=data.shape[0])
```
Результат работы равен `-0.006423534504746837`, а время работы составляет порядка 0.6с.

Ваша реализация должна удовлетворять следующим требованиям:
1. При вычислении не должно возникать warning, бесконечностей и nan-ов
2. Используйте не более одного цикла
3. Учтите, что метки кластеров могут идти не по порядку и принимать произвольные значения
4. Если в данных присутствует один кластер, то считайте что силуэт равен 0
5. Если $s_i = d_i = 0$, то $\text{sil}_i = 0$

# 4. Монотонная нелинейная функция

Рассмотрим произвольную монотонную непрерывную на отрезке $[-T, T]$ функцию $f(x)$. Ее можно приблизить кусочно-линейной непрерывной функцией $g(x)$ следующим образом. Построим равномерную сетку
$$ x_0=-T, x_1, x_2, \dots, x_{K-2}, x_{K-1} = T $$
на отрезке $[-T, T]$ с $K$ узлами. Пусть $g(-T) = b_0$ и $g(x)$ линейна на отрезках $[x_{i-1}, x_i]$. Поскольку $g(x)$ непрерывна, она однозначно задается значениями $b_0$ и коэффицентами наклона $\alpha_i$. Действительно,
$$ g(x_0) = b_0 $$
$$ g(x_i) = g(x_{i-1}) + \alpha_{i-1} (x_i - x_{i-1}), ~~ i=1,\dots,K-1 $$

При этом на практике неотрицательные углы наклона часто удобно параметризовать как
$$ \alpha_i = \log(1 + e^{v_i}) $$

Пусть задано значение $b_0$ и набор $v_i$, $i=0, 1, \dots, K-2$. Пусть также задан тензор $X$ значений $x_j$ произвольного размера. Гарантируется, что $-T \le x_j \le T$. Необходимо реализовать функцию, которая бы поэлементно вычисляла значения $g(X)$ (как это делают универсальные функции в numpy). Реализация не должна использовать циклов и сторонних библиотек, в том числе numba, `@vectorize` и прочее.

Для демонстрации правильности работы визуализируйте график функции с помощью matplotlib для нескольких разумных частных случаев.

Ориентировочное время работы для тензора $500 \times 500 \times 500$ и $K=1000$ равно 4.5с на 1 ядре.